In [3]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [4]:
df=pd.read_csv("C:/Users/nehas/OneDrive/Desktop/project/toxic/jigsaw-toxic-comment-classification-challenge/train.csv/train.csv")

In [5]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
from tensorflow.keras.layers import TextVectorization

In [7]:
x = df['comment_text']
y = df[df.columns [2:]].values

In [8]:
MAX_FEATURES = 200000

In [9]:
vectorizer = TextVectorization (max_tokens=MAX_FEATURES,
                                output_sequence_length=1800,
                                output_mode= 'int')

In [10]:
vectorizer.adapt(x.values)

In [11]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'to',
 'of',
 'and',
 'a',
 'you',
 'i',
 'is',
 'that',
 'in',
 'it',
 'for',
 'this',
 'not',
 'on',
 'be',
 'as',
 'have',
 'are',
 'your',
 'with',
 'if',
 'article',
 'was',
 'or',
 'but',
 'page',
 'my',
 'an',
 'from',
 'by',
 'do',
 'at',
 'about',
 'me',
 'so',
 'wikipedia',
 'can',
 'what',
 'there',
 'all',
 'has',
 'will',
 'talk',
 'please',
 'would',
 'its',
 'no',
 'one',
 'just',
 'like',
 'they',
 'he',
 'dont',
 'which',
 'any',
 'been',
 'should',
 'more',
 'we',
 'some',
 'other',
 'who',
 'see',
 'here',
 'also',
 'his',
 'think',
 'im',
 'because',
 'know',
 'how',
 'am',
 'people',
 'why',
 'edit',
 'articles',
 'only',
 'out',
 'up',
 'when',
 'were',
 'use',
 'then',
 'may',
 'time',
 'did',
 'them',
 'now',
 'being',
 'their',
 'than',
 'thanks',
 'even',
 'get',
 'make',
 'good',
 'had',
 'very',
 'information',
 'does',
 'could',
 'well',
 'want',
 'such',
 'sources',
 'way',
 'name',
 'these',
 'deletion',
 'pages',
 'first',
 'help'

In [12]:
vectorized_text= vectorizer (x.values)

In [13]:
vectorized_text


<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [14]:
dataset = tf.data.Dataset.from_tensor_slices ((vectorized_text, y))
dataset= dataset.cache()
dataset =dataset.shuffle (160000)
dataset =dataset.batch (16)
dataset =dataset.prefetch(8)

In [15]:
batch_X, batch_y =dataset.as_numpy_iterator ().next()

In [16]:
train = dataset.take(int(len (dataset)*.7))
val = dataset.skip(int(len (dataset)*.7)).take(int(len (dataset)*.2))
test = dataset.skip(int(len (dataset)*.9)). take(int(len (dataset)*.1))

In [17]:
train_generator=train.as_numpy_iterator()
train_generator.next()

(array([[90707,  4107, 93795, ...,     0,     0,     0],
        [  339,    16,    29, ...,     0,     0,     0],
        [ 2857,   171, 19616, ...,     0,     0,     0],
        ...,
        [    8,   243,  3292, ...,     0,     0,     0],
        [  151,    21,   737, ...,     0,     0,     0],
        [   21,   303,     9, ...,     0,     0,     0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dropout,Bidirectional,Dense,Embedding


In [19]:
model=Sequential()
model.add(Embedding(MAX_FEATURES+1,32))
model.add(Bidirectional(LSTM(32,activation='tanh')))
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(6,activation='sigmoid'))

In [20]:
model.compile(loss='BinaryCrossentropy',optimizer='adam')


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [22]:
history=model.fit(train,epochs=1,validation_data=val)

  11/6981 [..............................] - ETA: 3:13:57 - loss: 0.6155

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\nehas\AppData\Local\Temp\ipykernel_4364\1798384941.py", line 1, in <module>
    history=model.fit(train,epochs=1,validation_data=val)
  File "C:\Users\nehas\AppData\Roaming\Python\Python39\site-packages\keras\utils\traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "C:\Users\nehas\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1685, in fit
    tmp_logs = self.train_function(iterator)
  File "C:\Users\nehas\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\util\traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "C:\Users\nehas\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\eager\polymorphic_function\polymorphic_function.py", line 894, in __call__


TypeError: object of type 'NoneType' has no len()

In [ ]:
input_text=vectorizer('you are awefully bad! just makes me want to kill you')

In [ ]:
batch=test.as_numpy_iterator().next()

In [ ]:
batch_x,batch_y=test.as_numpy_iterator().next()


In [ ]:
(model.predict(batch_x)>0.5).astype(int)

In [ ]:
res=model.predict(np.expand_dims(input_text,0))

In [ ]:
from tensorflow.keras.metrics import Precision,Recall,CategoricalAccuracy

In [ ]:
pre=Precision()
re=Recall()
acc=CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    x_true,y_true=batch
    yhat=model.predict(x_true)
    y_true=y_true.flatten()
    yhat=yhat.flatten()
    pre.update_state(y_true,yhat)
    re.update_state(y_true,yhat)
    acc.update_state(y_true,yhat)
    

In [ ]:
print(f'Precision: {pre.result().numpy()},Recall:{re.result().numpy()},Accuracy:{acc.result().numpy()}')
    

In [ ]:
!pip install gradio jinja2

In [ ]:
import tensorflow as tf
import gradio as gr

In [ ]:
model.save('toxicity.h5')

In [ ]:
model=tf.keras.models.load_model('toxicity.h5')

In [ ]:
input_str=vectorizer("hey i hate you ! i am gonna kill you and shatter u")

In [ ]:
res=model.predict(np.expand_dims(input_str,0))

In [ ]:
res

In [ ]:
def score_comment(comment):
    vectorized_comment=vectorizer([comment])
    results=model.predict(vectorized_comment)
    text=""
    for idx, col in enumerate(df.columns[2:]):
        text+='{}: {}\n'.format(col,results[0][idx]>0.5)
    return text

In [ ]:
interface=gr.Interface(fn=score_comment,inputs=gr.inputs.Textbox(lines=2,placeholder='comment to score'),
                       outputs='text')

In [ ]:
interface.launch(share=True)
